In [1]:
from Bio.KEGG.KGML.KGML_parser import read
import pandas as pd
import requests
import re

In [4]:
pathway = read(open("hiv_raw_data/hsa04012.xml", 'r'))
print(pathway)

Pathway: ErbB signaling pathway
KEGG ID: path:hsa04012
Image file: https://www.kegg.jp/kegg/pathway/hsa/hsa04012.png
Organism: hsa
Entries: 80
Entry types:
	gene: 60
	group: 6
	compound: 4
	map: 10



In [9]:
entries_data = []
for entry in pathway.entries.values():
    entries_data.append({
        'id': entry.id,
        'name': entry.name,
        'type': entry.type,
        'link': entry.link,
        'reaction': entry.reaction,
    })
entries_df = pd.DataFrame(entries_data)
len(entries_df)

80

In [6]:
entries_df.head()

,id,name,type,link,reaction
0,7,path:hsa05223,map,https://www.kegg.jp/dbget-bin/www_bget?hsa05223,
1,8,hsa:2885,gene,https://www.kegg.jp/dbget-bin/www_bget?hsa:2885,
2,9,hsa:2002,gene,https://www.kegg.jp/dbget-bin/www_bget?hsa:2002,
3,10,hsa:2066,gene,https://www.kegg.jp/dbget-bin/www_bget?hsa:2066,
4,11,hsa:2066,gene,https://www.kegg.jp/dbget-bin/www_bget?hsa:2066,


In [7]:
set(entries_df['type'])

{'compound', 'gene', 'group', 'map'}

In [ ]:
proteins_df = entries_df[entries_df['type'] == 'ortholog'] 